In [65]:
import pymysql
import getpass as gp
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [66]:
# conn1 = pymysql.connect(host='127.0.0.1', user='root', password = input(), db='employees', charset='utf8mb4')
# conn2 = pymysql.connect(host='127.0.0.1', user='reader', password = input(), db='employees', charset='utf8mb4')

In [67]:
# 계정 정보 사전
accounts = {
    'root': {'host': '127.0.0.1', 'db': 'employees'},
    'reader': {'host': '127.0.0.1', 'db': 'employees'},
    'writer': {'host': '127.0.0.1', 'db': 'employees'},
    'edward': {'host': '192.168.0.27', 'db': 'employees'},
    # 여기에 계정 추가 가능
}

def connect_to_db():
    user = input("ID: ").strip()
    if user not in accounts:
        raise ValueError("등록되지 않은 사용자 ID")

    password = gp.getpass("비밀번호: ")
    info = accounts[user]

    conn = pymysql.connect(
        host=info['host'],
        user=user,
        password=password,
        #db=info['db'],
        charset='utf8mb4'
    )
    return conn

def q(query):
    with conn.cursor() as cursor: # 커서 생성 with 구문을 사용해서 자동으로 close 하므로 메모리 누수를 방지
        cursor.execute(query)
        first = query.strip().split()[0].lower()
        if first in ['select', 'show', 'describe', 'desc', 'explain']:
            df = pd.read_sql(query, conn)
            display(df)
        else:
            conn.commit()
            print("Query OK.")


In [68]:
conn = connect_to_db()

In [69]:
q('USE employees')

Query OK.


In [70]:
q("SELECT current_role();")

,current_role()
0,NONE


In [71]:
q("SELECT user, host, account_locked FROM mysql.user;")

,user,host,account_locked
0,role_emp_read,%,Y
1,role_emp_write,%,Y
2,reader,127.0.0.1,N
3,writer,127.0.0.1,N
4,edward,192.168.0.28,N
5,mysql.infoschema,localhost,Y
6,mysql.session,localhost,Y
7,mysql.sys,localhost,Y
8,root,localhost,N


In [72]:
q("SELECT user, host, account_locked FROM mysql.user WHERE account_locked = 'Y';")

,user,host,account_locked
0,role_emp_read,%,Y
1,role_emp_write,%,Y
2,mysql.infoschema,localhost,Y
3,mysql.session,localhost,Y
4,mysql.sys,localhost,Y


In [73]:
q("SELECT user, host, account_locked FROM mysql.user WHERE account_locked = 'N';")

,user,host,account_locked
0,reader,127.0.0.1,N
1,writer,127.0.0.1,N
2,edward,192.168.0.28,N
3,root,localhost,N


| 구문                 | 재실행 시 영향            | 설명                                                            |
| ------------------ | ------------------- | ------------------------------------------------------------- |
| `INSERT INTO`      | ❗**데이터 중복 삽입**      | 같은 레코드 계속 추가됨                                                 |
| `UPDATE`           | ❗**데이터 덮어쓰기**       | 조건 잘못 걸면 의도치 않게 전체 업데이트                                       |
| `DELETE`           | ❗**데이터 삭제**         | 조건 실수하면 전체 삭제 가능                                              |
| `TRUNCATE TABLE`   | ❗**모든 데이터 삭제**      | 되돌릴 수 없음 (DML 아님)                                             |
| `DROP TABLE`       | ❗**테이블 자체 삭제**      | 데이터+구조 완전 삭제됨                                                 |
| `DROP DATABASE`    | ❗**DB 전체 삭제**       | 되돌릴 수 없음                                                      |
| `CREATE TABLE`     | ⚠️ **에러 또는 덮어씀**    | `IF NOT EXISTS` 안 쓰면 에러. `LIKE`, `AS SELECT` 형태는 구조 복제 가능성 주의 |
| `ALTER TABLE`      | ⚠️ **구조 덮어쓰기/변경**   | 컬럼 추가/변경 시 기존 데이터 손상 위험                                       |
| `LOAD DATA INFILE` | ❗**대량 삽입**          | 실수로 중복 데이터 대량 삽입 가능                                           |
| `REPLACE INTO`     | ❗**기존 데이터 삭제 후 삽입** | `PRIMARY KEY` 충돌 시 기존 데이터 삭제됨                                 |

위 구문들은 한번 사용하면 주석 처리해서 중복사용 안되게 조심해서 실행

In [74]:
q("""
CREATE TABLE IF NOT EXISTS users (
    id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(50),
    age INT
);
""")
# q("INSERT INTO users (name, age) VALUES ('Alice', 25), ('Bob', 30);")
q("SELECT * FROM users;")


Query OK.


,id,name,age


In [75]:
q('SELECT DATABASE();')

,DATABASE()
0,employees


In [76]:
q("SHOW TABLES;")

,Tables_in_employees
0,departments
1,dept_emp
2,dept_manager
3,employee_docs
4,employee_name
5,employees
6,employees_comp4k
7,employees_comp8k
8,salaries
9,tb_dual


In [77]:
q("DESCRIBE users;")

,Field,Type,Null,Key,Default,Extra
0,id,int,NO,PRI,None,auto_increment
1,name,varchar(50),YES,,None,
2,age,int,YES,,None,


In [78]:
q("SELECT * FROM users;")

,id,name,age


In [79]:
q("DROP TABLE IF EXISTS users;")

Query OK.


In [80]:
q('CREATE DATABASE IF NOT EXISTS test;')

Query OK.


In [81]:
q("SHOW DATABASES;")

,Database
0,employees
1,information_schema
2,mysql
3,performance_schema
4,sys
5,test
6,testdb


In [82]:
q("USE test;")
q("SELECT DATABASE();")

Query OK.


,DATABASE()
0,test


In [83]:
force_data = pd.read_csv("C:/Users/Edward/Downloads/force_data.csv")

In [84]:
force_data

,sensor_image,F_mag,F_x,F_y,F_z
0,image_1.png,0.862067,-0.794130,-0.031904,-0.333917
1,image_2.png,0.696529,-0.640995,-0.055059,-0.266921
2,image_3.png,1.046410,-0.999519,-0.012941,-0.309467
3,image_4.png,0.946791,-0.889747,0.021104,-0.322983
4,image_5.png,1.458857,-1.416761,-0.028729,-0.346736
...,...,...,...,...,...
3995,image_3996.png,0.212787,-0.156948,0.031501,-0.140190
3996,image_3997.png,0.327715,-0.260379,0.063919,-0.188452
3997,image_3998.png,0.380441,-0.326094,0.020620,-0.194867
3998,image_3999.png,0.734159,-0.715075,0.007800,-0.166123


In [85]:
class AutoTable:
    def __init__(self, conn):
        self.conn = conn

    def infer_sql_type(self, series):
        dtype = str(series.dtype)

        if dtype == 'object':
            max_len = series.dropna().astype(str).str.len().max()
            if pd.isna(max_len):  # 모두 NaN인 경우
                return 'VARCHAR(255)'
            return f'VARCHAR({max_len if max_len > 0 else 1})'

        if dtype == 'int64':
            return 'INT'
        if dtype == 'float64':
            return 'FLOAT'
        if dtype == 'bool':
            return 'BOOLEAN'
        if 'datetime' in dtype:
            return 'DATETIME'

        return 'TEXT'  # fallback

    def create_table(self, df, table_name):
        columns_sql = []

        primary_key = None
        for col in df.columns:
            col_sql = f"`{col}`"
            series = df[col]
            sql_type = self.infer_sql_type(series)

            # 자동 PRIMARY KEY + AUTO_INCREMENT 감지
            if col.lower() == 'id' and pd.api.types.is_integer_dtype(series):
                if series.is_monotonic_increasing and series.is_unique:
                    col_sql += f" {sql_type} PRIMARY KEY AUTO_INCREMENT"
                    primary_key = col
                else:
                    col_sql += f" {sql_type}"
            else:
                col_sql += f" {sql_type}"

            # NOT NULL 감지
            if not series.isnull().any():
                col_sql += " NOT NULL"

            columns_sql.append(col_sql)

        columns_sql_str = ',\n  '.join(columns_sql)
        create_stmt = f"CREATE TABLE IF NOT EXISTS `{table_name}` (\n  {columns_sql_str}\n);"

        with self.conn.cursor() as cursor:
            cursor.execute(create_stmt)
        self.conn.commit()
        print(f"`{table_name}` 테이블 생성 완료")

    def insert_data(self, df, table_name):
        col_names = ', '.join(f"`{col}`" for col in df.columns)
        placeholders = ', '.join(['%s'] * len(df.columns))
        insert_stmt = f"INSERT INTO `{table_name}` ({col_names}) VALUES ({placeholders})"

        with self.conn.cursor() as cursor:
            for row in df.itertuples(index=False):
                cursor.execute(insert_stmt, tuple(row))
        self.conn.commit()
        print(f"`{table_name}`에 {len(df)}개 행 삽입 완료")

    def create_and_insert(self, df, table_name):
        self.create_table(df, table_name)
        self.insert_data(df, table_name)


In [86]:
at2 = AutoTable(conn)
at2.create_and_insert(force_data, 'force_data')


`force_data` 테이블 생성 완료
`force_data`에 4000개 행 삽입 완료


In [87]:
q("SELECT * FROM targets;")

,sensor_image,object_label,pose_x,pose_y,pose_z,pose_Rx,pose_Ry,pose_Rz,shear_x,shear_y,shear_z,shear_Rx,shear_Ry,shear_Rz,object_x,object_y,object_z,object_Rx,object_Ry,object_Rz
0,image_1.png,edge,0.0,-4.18311,-31.6896,0.0,0.0,-0.539135,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0
1,image_2.png,edge,0.0,-2.95067,-34.9233,0.0,0.0,-2.294220,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0
2,image_3.png,edge,0.0,-4.88164,-21.9066,0.0,0.0,2.116500,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0
3,image_4.png,edge,0.0,-3.92630,-41.7070,0.0,0.0,-0.468825,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0
4,image_5.png,edge,0.0,-5.26053,-47.0288,0.0,0.0,2.221410,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,image_3996.png,edge,0.0,-1.34567,-30.0608,0.0,0.0,0.181993,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0
3996,image_3997.png,edge,0.0,-1.26415,-45.1295,0.0,0.0,1.073750,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0
3997,image_3998.png,edge,0.0,-1.89177,-16.2608,0.0,0.0,-0.482602,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0
3998,image_3999.png,edge,0.0,-3.07385,-12.2836,0.0,0.0,-0.195525,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0


In [88]:
q("SELECT * FROM force_data;")

,sensor_image,F_mag,F_x,F_y,F_z
0,image_1.png,0.862067,-0.794130,-0.031904,-0.333917
1,image_2.png,0.696529,-0.640995,-0.055059,-0.266921
2,image_3.png,1.046410,-0.999519,-0.012941,-0.309467
3,image_4.png,0.946791,-0.889747,0.021104,-0.322982
4,image_5.png,1.458860,-1.416760,-0.028729,-0.346736
...,...,...,...,...,...
15995,image_3996.png,0.212787,-0.156948,0.031501,-0.140190
15996,image_3997.png,0.327715,-0.260379,0.063919,-0.188452
15997,image_3998.png,0.380441,-0.326094,0.020620,-0.194867
15998,image_3999.png,0.734159,-0.715075,0.007800,-0.166123


In [ ]:
q("""
# CREATE TABLE JOIN
SELECT *
FROM targets t
JOIN force_data f
ON t.sensor_image = f.sensor_image

""")


ProgrammingError: (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'JOIN AS\nSELECT *\nFROM targets t\nJOIN force_data f\nON t.sensor_image = f.sensor_i' at line 1")

In [ ]:
# q("""
# CREATE VIEW IF  JOINED_VIEW AS
# SELECT 
#   t.*,
#   f.F_mag, f.F_x, f.F_y, f.F_z
# FROM targets t
# JOIN force_data f
#   ON t.sensor_image = f.sensor_image;
# """)

In [ ]:
q("SELECT * FROM JOINED_VIEW;")

,sensor_image,object_label,pose_x,pose_y,pose_z,pose_Rx,pose_Ry,pose_Rz,shear_x,shear_y,...,object_x,object_y,object_z,object_Rx,object_Ry,object_Rz,F_mag,F_x,F_y,F_z
0,image_1.png,edge,0.0,-4.18311,-31.6896,0.0,0.0,-0.539135,0.0,0.0,...,0,0,0,0,0,0,0.862067,-0.794130,-0.031904,-0.333917
1,image_2.png,edge,0.0,-2.95067,-34.9233,0.0,0.0,-2.294220,0.0,0.0,...,0,0,0,0,0,0,0.696529,-0.640995,-0.055059,-0.266921
2,image_3.png,edge,0.0,-4.88164,-21.9066,0.0,0.0,2.116500,0.0,0.0,...,0,0,0,0,0,0,1.046410,-0.999519,-0.012941,-0.309467
3,image_4.png,edge,0.0,-3.92630,-41.7070,0.0,0.0,-0.468825,0.0,0.0,...,0,0,0,0,0,0,0.946791,-0.889747,0.021104,-0.322982
4,image_5.png,edge,0.0,-5.26053,-47.0288,0.0,0.0,2.221410,0.0,0.0,...,0,0,0,0,0,0,1.458860,-1.416760,-0.028729,-0.346736
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11995,image_3987.png,edge,0.0,-3.32458,-45.0680,0.0,0.0,-1.230850,0.0,0.0,...,0,0,0,0,0,0,0.685950,-0.642747,0.000445,-0.239592
11996,image_3991.png,edge,0.0,-3.02210,-32.0665,0.0,0.0,0.702956,0.0,0.0,...,0,0,0,0,0,0,0.456609,-0.393377,0.059042,-0.224188
11997,image_3994.png,edge,0.0,-4.54282,-21.0129,0.0,0.0,-1.744030,0.0,0.0,...,0,0,0,0,0,0,0.737330,-0.712031,0.029768,-0.189156
11998,image_3995.png,edge,0.0,-2.39519,-12.4319,0.0,0.0,2.315990,0.0,0.0,...,0,0,0,0,0,0,0.546032,-0.537622,0.004972,-0.095334


In [ ]:
q("""
  UPDATE targets
  SET sensor_image = 'image_4000.jpg'
  WHERE object_label = 'a';
  """)

Query OK.
